In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # Used to fix missing values


Automobile = pd.read_excel('Automobile.xlsx') 

print("Initial Missing Values:\n", Automobile.isnull().sum())

# --- 2. Data Cleaning - Missing Values (CRITICAL FIX) ---
# Use SimpleImputer from sklearn to fill NaN values with the median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# We only fit and transform the 'horsepower' column
Automobile['horsepower'] = imputer.fit_transform(Automobile[['horsepower']])

print("\nMissing Values After Imputation:\n", Automobile.isnull().sum())


#3. Data Cleaning - Duplicates 
before = Automobile.shape[0]
Automobile.drop_duplicates(inplace=True)
after = Automobile.shape[0]
print(f"\nRemoved {before - after} duplicate rows.")


#4. Data Cleaning - Categorical Encoding (CRITICAL FIX)
# The correct categorical column is 'origin', not 'horsepower'.
# Drop 'name' as it's an identifier and not useful for modeling.
Automobile_encoded = pd.get_dummies(Automobile.drop('name', axis=1), 
                                     columns=['origin'], 
                                     prefix='origin',
                                     drop_first=True) # drop_first avoids multicollinearity

print("\nEncoded Data Head (Origin Encoded):\n", Automobile_encoded.head())


#5. Feature Scaling (Standardization)
# Select the features for scaling (now including the imputed 'horsepower')
features_to_scale = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']

# Initialize and apply the scaler (StandardScaler needs to be imported)
scaler_standard = StandardScaler()

# Here we apply it only to the specified numerical features.
df_standard_scaled = pd.DataFrame(scaler_standard.fit_transform(Automobile_encoded[features_to_scale]), 
                                  columns=features_to_scale)

# Re-integrate the scaled features into the full cleaned DataFrame
for col in features_to_scale:
    Automobile_encoded[col] = df_standard_scaled[col]

print("\nDataset Head After Scaling:\n", Automobile_encoded.head())


#6. Outlier Detection (Correctly implemented, but not treating them)
Q1 = Automobile_encoded.select_dtypes(include=['number']).quantile(0.25)
Q3 = Automobile_encoded.select_dtypes(include=['number']).quantile(0.75)
IQR = Q3 - Q1
outliers = ((Automobile_encoded.select_dtypes(include=['number']) < (Q1 - 1.5 * IQR)) | 
            (Automobile_encoded.select_dtypes(include=['number']) > (Q3 + 1.5 * IQR)))
print("\nTotal Outliers by Column (Sum):")
print(outliers.sum())


#7. Save Final Cleaned File
Automobile_encoded.to_excel("Cleaned_file.xlsx", index=False)


Initial Missing Values:
 name            0
mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64

Missing Values After Imputation:
 name            0
mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64

Removed 0 duplicate rows.

Encoded Data Head (Origin Encoded):
     mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year  origin_japan  origin_usa  
0          70         False        True  
1          70       